In [ ]:
#deal 번호를 받아와서
#http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/4284526834?&page=100&count=100
#reviewContents가 없을 때까지

In [ ]:
import urllib
from bs4 import BeautifulSoup
import requests
import json
import re

def toJson(data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(data, file)
    return 

# txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt', 'productUrlList_과일.txt']
txt_file_path_ls = ['test.txt']

for txt_file_path in txt_file_path_ls :

    #productUrlList.txt에서 링크 리스트 불러오기
    r = open(txt_file_path, mode='rt', encoding='utf-8')
    productUrlList = r.readlines()

    review_data = []

    for prod_url in productUrlList :
        prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
        prod_idx = re.sub('\n','',prod_idx)
        
        page = 1
        while True: 
            url = "http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/{}?&page={}&count=100".format(prod_idx, page)
            html = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(html, 'html.parser')
            text = soup.get_text()
            json_data = json.loads(text)


            try :
                iter_length = len(json_data['data']['reviewContents'])

                for idx in range(iter_length) :

                    #리뷰 고유 번호
                    review_idx  = json_data['data']['reviewContents'][idx]['reviewSrl']
                    #상품 번호
                    prod_idx = deal
                    #상품 만족도
                    prod_satis = json_data['data']['reviewContents'][idx]['dealGpoint']
                    #배송 만족도
                    ship_satis = json_data['data']['reviewContents'][idx]['deliveryGpoint']
                    #구매한 옵션
                    select_options = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['optionTitle']
                    #구매한 날짜
                    buy_date = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['optionTitle']
                    #리뷰 작성 날짜
                    review_date = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['updateDt']
                    #사용만족도 옵션1
                    use_opt1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['keyword']
                    use_satis1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['selection']
                    #사용만족도 옵션2
                    use_opt2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['keyword']
                    use_satis2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['selection']
                    #리뷰 내용
                    content = json_data['data']['reviewContents'][idx]['contents']
                    #user_id
                    user_id = json_data['data']['reviewContents'][idx]['whoUpdate']
                    #memberSrl
                    memberSrl =json_data['data']['reviewContents'][idx]['memberSrl']

                    review_data.append({
                            'review_idx' : review_idx,
                            'prod_idx' : prod_idx,
                            'prod_satis' : prod_satis,
                            'ship_satis' : ship_satis,
                            'select_options' : select_options,
                            'buy_date' : buy_date,
                            'review_date' : review_date,
                            'use_opt1' : use_opt1,
                            'use_satis1' : use_satis1,
                            'use_opt2' : use_opt2,
                            'use_satis2' : use_satis2,
                            'content' : content,
                            'user_id' : user_id,
                            'memberSrl' : memberSrl
                        })
            except :
                break

            page += 1

    toJson(review_data, 'review_data_{}.json'.format(txt_file_path[15:-4]))

In [ ]:
data_df = pd.read_json('review_data_.json')
data_df